# Model Agent (Maze Solver)

The environment I have chosen here is a maze which has one entry and one exit. The goal of the agent is to get to reach the final exit location from the starting location. The agent has no idea of the location of the exit, As this is a model based agent so I designed it such that it keep track of the last location ((x, y) position) where it had to make a choice to go left or right or up or down and stores it in list. Whenever the agent encounters a dead-end or is stuck somewhere then it will go back to that location and move to the other side. I made the memory like a dictionary where key is the (x, y) location and value are the possible moves availabe and not performed yet. So if it reaches that place again then it will choose from the remaining moves or make a new entry in that memory if it came there for the first time.

In [1]:
import random
import numpy as np

In [2]:
'''
This class is made to either take in a given maze in an array form along with the max limit and goal state or just
randomly make a maze (It does not necessarily have to be a functional maze) which might not have a path too but it useful
to test if our agent and know of it will react and which action it will take. I have made a seperate file for the maze
generation algorithm.
'''

class maze_environment:
    def __init__(self, limit=None, maze=None, goal=None):
        if(maze is None and limit is None):
            self.limit = random.randint(5, 15)
            self.goal = (self.limit-1, self.limit-1)
            self.maze = []
            for i in range(self.limit):
                row = []
                for j in range(self.limit):
                    row.append(random.choice(['F', 'B']))
                self.maze.append(row)
            self.maze[0][0] = 'F'
            self.maze[self.limit-1][self.limit-1] = 'F'
        else:
            self.maze = maze
            self.limit = limit
            self.goal = goal
    
    def get_limit(self):
        return self.limit
    
    def show_maze(self):
        for i in range(self.limit):
            for j in range(self.limit):
                print(self.maze[i][j], end=" ")
            print()
    
    def get_goal(self):
        return self.goal
    
    def get_position(self, x, y):
        if(x > self.limit-1 or y > self.limit-1):
            return 'B'
        if(x < 0 or y < 0):
            return 'B'
        return self.maze[x][y]

In [3]:
class maze_solver_full_memory:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.total_steps = 0
        self.prev_move = -1
        self.branches = {}
    
    def tell_pos(self):
        print("I am currently at the position ({}, {})".format(self.x, self.y))
    
    def move(self, env):
        if(self.x == env.get_goal()[0] and self.y == env.get_goal()[1]):
            print("You have solved the maze and reached the end...")
            return False
        elif(self.total_steps >= env.limit*env.limit):
            print("Did not halt, mostly unsolvable maze...")
            return False
        else:
            self.surrondings = [env.get_position(self.x+1, self.y),
                                env.get_position(self.x, self.y+1),
                                env.get_position(self.x-1, self.y), 
                                env.get_position(self.x, self.y-1)]
            possible_moves = []
            for i in range(len(self.surrondings)):
                if self.surrondings[i] == 'F':
                    possible_moves.append(i)
                    
            if(self.prev_move == 0):
                if(2 in possible_moves):
                    possible_moves.remove(2)
            elif(self.prev_move == 2):
                if(0 in possible_moves):
                    possible_moves.remove(0)
            elif(self.prev_move == 1):
                if(3 in possible_moves):
                    possible_moves.remove(3)
            elif(self.prev_move == 3):
                if(1 in possible_moves):
                    possible_moves.remove(1)
                    
            move_turn = 1
            if(len(possible_moves) == 1):
               current_move = possible_moves[0]
            elif(len(possible_moves) > 1):
                if((self.x, self.y) not in self.branches):
                    current_move = random.choice(possible_moves)
                    possible_moves.remove(current_move)
                    self.branches[(self.x, self.y)] = possible_moves
                else:
                    if(self.branches[(self.x, self.y)]):
                        current_move = random.choice(self.branches[(self.x, self.y)])
                        self.branches[(self.x, self.y)].remove(current_move)
                    else:
                        current_move = random.choice(possible_moves)
                        possible_moves.remove(current_move)
                        self.branches[(self.x, self.y)] = possible_moves
            else:
                if(not self.branches):
                    print("Unsolvable maze, no route to exit...")
                    return False
                last_choice_position = self.branches.popitem()
                self.x = last_choice_position[0][0]
                self.y = last_choice_position[0][1]
                self.branches[(self.x, self.y)] = last_choice_position[1]
                self.total_steps = 0
                move_turn = 0
            
            if(move_turn):
                if(current_move == 0):
                    self.x += 1
                    self.prev_move = 0
                    print("Go Down")
                elif(current_move == 1):
                    self.y += 1
                    self.prev_move = 1
                    print("Go Right")
                elif(current_move == 2):
                    self.x -= 1
                    self.prev_move = 2
                    print("Go Up")
                elif(current_move == 3):
                    self.y -= 1
                    self.prev_move = 3
                    print("Go Left")
                else:
                    print("Maze is blocked for all the places...")
                    return False
                
            self.total_steps += 1
            return True

### Running the agent in various custom environments to see how it performs 

In [4]:
#Custom maze 1 - Single path to goal
cust_maze = [['F', 'B', 'B', 'B', 'F'],
             ['F', 'B', 'B', 'B', 'B'],
             ['F', 'F', 'F', 'B', 'B'],
             ['B', 'B', 'F', 'F', 'F'],
             ['B', 'B', 'B', 'B', 'F']]

In [5]:
my_env = maze_environment(limit=5, maze=cust_maze, goal=(4, 4))
my__agent = maze_solver_full_memory()

print("Limit is: ", my_env.get_limit())
print("Goal is: ", my_env.get_goal())
my_env.show_maze()

Limit is:  5
Goal is:  (4, 4)
F B B B F 
F B B B B 
F F F B B 
B B F F F 
B B B B F 


In [6]:
status = True
while(status):
    status = my__agent.move(my_env)
    my__agent.tell_pos()

Go Down
I am currently at the position (1, 0)
Go Down
I am currently at the position (2, 0)
Go Right
I am currently at the position (2, 1)
Go Right
I am currently at the position (2, 2)
Go Down
I am currently at the position (3, 2)
Go Right
I am currently at the position (3, 3)
Go Right
I am currently at the position (3, 4)
Go Down
I am currently at the position (4, 4)
You have solved the maze and reached the end...
I am currently at the position (4, 4)


In [7]:
cust_maze2 = [['F', 'F', 'F', 'B', 'B'],
              ['F', 'F', 'B', 'B', 'F'],
              ['F', 'B', 'B', 'B', 'B'],
              ['F', 'B', 'F', 'F', 'F'],
              ['F', 'F', 'F', 'B', 'F']]

In [8]:
my_env2 = maze_environment(limit=5, maze=cust_maze2, goal=(4, 4))
my_agent2 = maze_solver_full_memory()

print("Limit is: ", my_env2.get_limit())
print("Goal is: ", my_env2.get_goal())
my_env2.show_maze()

Limit is:  5
Goal is:  (4, 4)
F F F B B 
F F B B F 
F B B B B 
F B F F F 
F F F B F 


In [9]:
status = True
while(status):
    status = my_agent2.move(my_env2)
    my_agent2.tell_pos()

Go Right
I am currently at the position (0, 1)
Go Down
I am currently at the position (1, 1)
Go Left
I am currently at the position (1, 0)
Go Down
I am currently at the position (2, 0)
Go Down
I am currently at the position (3, 0)
Go Down
I am currently at the position (4, 0)
Go Right
I am currently at the position (4, 1)
Go Right
I am currently at the position (4, 2)
Go Up
I am currently at the position (3, 2)
Go Right
I am currently at the position (3, 3)
Go Right
I am currently at the position (3, 4)
Go Down
I am currently at the position (4, 4)
You have solved the maze and reached the end...
I am currently at the position (4, 4)
